## 테스트

In [1]:
# Identifying numerical and categorical columns
numerical_cols = train.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = train.select_dtypes(include=['object']).columns

# Preprocessing for numerical data
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()) 
])

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define the model
model = DecisionTreeRegressor()

# Create a pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', model)])

# Splitting data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(train, y, test_size=0.2, random_state=0)

# Define the parameter grid for RandomizedSearchCV
param_dist = {
    'regressor__max_depth': np.concatenate((np.arange(3, 11), [None])),  # Range from 3 to 10, and None
    'regressor__min_samples_split': np.arange(2, 11),  # Range from 2 to 10
    'regressor__min_samples_leaf': np.arange(1, 5)  # Range from 1 to 4
}

# RandomizedSearchCV
random_search = RandomizedSearchCV(pipeline, param_distributions=param_dist, n_iter=10, cv=5, random_state=0)
random_search.fit(X_train, y_train)

# Best parameters
print("Best parameters:", random_search.best_params_)

Best parameters found:  {'classifier__min_samples_split': 2, 'classifier__min_samples_leaf': 2, 'classifier__max_depth': 5}
Best parameters found:  {'classifier__min_samples_split': 2, 'classifier__min_samples_leaf': 2, 'classifier__max_depth': 5}
Accuracy:  0.6938775510204082
Precision:  0.6923076923076923
Recall:  0.9
F1 Score:  0.7826086956521738


In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np

y_val_pred = np.round(random_search.predict(X_val), 0)

rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
print(f"RMSE on Validation Set: {rmse}")

In [ ]:
# Preprocess and predict on the test set
test_pred = np.round(random_search.predict(test), 0).astype('int64')
submission_df = pd.DataFrame({'Id': test['Id'], 'SalePrice': np.round(test_pred, 0)})
submission_df.to_csv('output/submission.csv', index=False)